<a href="https://colab.research.google.com/github/opethaiwoh/MovieReview/blob/main/PySpark_TextMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Text Mining Analysis of Movie Dataset**<br>



In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
#Always check for the updates on: https://apache.osuosl.org/spark
!wget -q https://apache.osuosl.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop2.7.tgz

^C


In [3]:
#also change the version of the spark to e.g 3.2.4
!tar xf spark-3.2.4-bin-hadoop2.7.tgz

In [4]:
pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#change the spark version here too
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop2.7"

In [6]:
os.environ["SPARK_HOME"]

'/content/spark-3.2.4-bin-hadoop2.7'

You don't need to install the latest version

In [7]:
pip install spark-nlp==2.4.2

Install the version of spark

In [8]:
pip install pyspark==3.2.4

In [9]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.4.2
Apache Spark version:  3.2.4


In [10]:
pip install langid

In [10]:
pip install nltk

In [11]:
import nltk

In [12]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [13]:
import string

In [14]:
# 14.2. Text Preprocessing
# NOTE: Rerun from here if you get SparkContexts error.
def check_blanks(data_str):
    is_blank = str(data_str.isspace())
    return is_blank

In [15]:
# Remove features
import re
import string
def remove_features(data_str):
    # compile regex
    url_re = re.compile('https?://(www.)?\w+\.\w+(/\w+)*/?')
    punc_re = re.compile('[%s]' % re.escape(string.punctuation))
    punc_re = re.compile('[^A-Za-z0-9]+')
    num_re = re.compile('(\\d+)')
    mention_re = re.compile('@(\w+)')
    alpha_num_re = re.compile("^[a-z0-9_.]+$")
    # convert to lowercase
    data_str = data_str.lower()
    # remove hyperlinks
    data_str = url_re.sub(' ', data_str)
    # remove @mentions
    data_str = mention_re.sub(' ', data_str)
    # remove puncuation
    data_str = punc_re.sub(' ', data_str)
    # remove numeric 'words'
    data_str = num_re.sub(' ', data_str)
    # remove non a-z 0-9 characters and words shorter than 3 characters
    list_pos = 0
    cleaned_str = ''
    for word in data_str.split():
        if list_pos == 0:
            if alpha_num_re.match(word) and len(word) > 2:
                cleaned_str = word
            else:
                cleaned_str = ' '
        else:
            if alpha_num_re.match(word) and len(word) > 2:
                cleaned_str = cleaned_str + ' ' + word
            else:
                cleaned_str += ' '
        list_pos += 1
    return cleaned_str

In [16]:
# Removes stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords.words('english')
print(stopwords.words('english'))
#print(stops)
def remove_stops(data_str):
    # expects a string
    stopwords = nltk.corpus.stopwords.words('english')
    list_pos = 0
    cleaned_str = ''
    text = data_str.split()
    for word in text:
        if word not in stopwords:
            # rebuild cleaned_str
            if list_pos == 0:
                cleaned_str = word
            else:
                cleaned_str = cleaned_str + ' ' + word
            list_pos += 1
    return cleaned_str

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# Tagging text
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
def tag_and_remove(data_str):
    cleaned_str = ' '
    # noun tags
    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
    # adjectives
    jj_tags = ['JJ', 'JJR', 'JJS']
    # verbs
    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    nltk_tags = nn_tags + jj_tags + vb_tags

    # break string into 'words'
    text = data_str.split()

    # tag the text and keep only those with the right tags
    tagged_text = nltk.pos_tag(text)
    for tagged_word in tagged_text:
        if tagged_word[1] in nltk_tags:
            cleaned_str += tagged_word[0] + ' '

    return cleaned_str

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [18]:
# Lemmatization
import re
import string
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
def lemmatize(data_str):
    # expects a string
    list_pos = 0
    cleaned_str = ''
    lmtzr = WordNetLemmatizer()
    text = data_str.split()
    tagged_words = nltk.pos_tag(text)
    for word in tagged_words:
        if 'v' in word[1].lower():
            lemma = lmtzr.lemmatize(word[0], pos='v')
        else:
            lemma = lmtzr.lemmatize(word[0], pos='n')
        if list_pos == 0:
            cleaned_str = lemma
        else:
            cleaned_str = cleaned_str + ' ' + lemma
        list_pos += 1
    return cleaned_str

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Create a SparkSession
spark = SparkSession.builder.appName("MovieRv").getOrCreate()

# Define your UDFs here
remove_stops_udf = udf(remove_stops, StringType())
remove_features_udf = udf(remove_features, StringType())
tag_and_remove_udf = udf(tag_and_remove, StringType())
lemmatize_udf = udf(lemmatize, StringType())
check_blanks_udf = udf(check_blanks, StringType())

# Load your data using SparkSession
neg_rdd = spark.sparkContext.textFile("/content/drive/Shareddrives/Black in AI/Dataset/combine_neg.txt")
pos_rdd = spark.sparkContext.textFile("/content/drive/Shareddrives/Black in AI/Dataset/combine_pos.txt")
neg_parts_rdd = neg_rdd.map(lambda l: l.split("\t"))
pos_parts_rdd = neg_rdd.map(lambda l: l.split("\t"))
neg_filled_rdd = neg_parts_rdd.filter(bool)
pos_filled_rdd = pos_parts_rdd.filter(bool)
neg_data_df = spark.createDataFrame(neg_filled_rdd, StringType())
pos_data_df = spark.createDataFrame(pos_filled_rdd, StringType())

# Print the schema of DataFrames
neg_data_df.printSchema()
pos_data_df.printSchema()


root
 |-- value: string (nullable = true)

root
 |-- value: string (nullable = true)



In [25]:
import nltk
from nltk.corpus import stopwords
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Download NLTK stopwords (only once)
nltk.download('stopwords')

# Define your UDFs here (replace this with your actual UDF definitions)
def remove_stops(text):
    # Define your logic to remove stopwords
    stopwords_list = stopwords.words('english')
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords_list]
    return ' '.join(filtered_words)

remove_stops_udf = udf(remove_stops, StringType())

# Define the columns to select from the DataFrame
neg_raw_cols = ["value"]

# Apply the UDF to remove stopwords
neg_rm_stops_df = neg_data_df.select(neg_raw_cols).withColumn("neg_stop_text", remove_stops_udf(neg_data_df["value"]))
neg_rm_stops_df.show(20)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


+--------------------+--------------------+
|               value|       neg_stop_text|
+--------------------+--------------------+
|[I can only say t...|[I say this: ee03...|
|[.maybe.Though I ...|[.maybe.Though I ...|
|[..well it doesn'...|[..well really on...|
|[and I haven't ev...|[and I even got f...|
|[ or will he just...|[ waste amounts m...|
|[ this is fine."<...|[ fine."<br /><br...|
|[ there was "Raw,...|[ "Raw," really s...|
|[ <br /><br />"Su...|[ <br /><br />"Su...|
|[although it was ...|[although actuall...|
|                  []|                  []|
|[.at least it wou...|[.at least would ...|
|[and the action s...|[and action shot ...|
|[but then again, ...|[but again, they'...|
|[he meant well. A...|[he meant well. A...|
|[but we watched i...|[but watched it, ...|
|[ just another un...|[ another unsolve...|
|[". After a coupl...|[". After couple ...|
|[but judge that f...|[but judge yourse...|
|[ but actually it...|[ actually kind a...|
|          [ kind of]|          

In [26]:
import nltk
from nltk.corpus import stopwords
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Download NLTK stopwords (only once)
nltk.download('stopwords')

# Define your UDFs here (replace this with your actual UDF definitions)
def remove_stops(text):
    # Define your logic to remove stopwords
    stopwords_list = stopwords.words('english')
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords_list]
    return ' '.join(filtered_words)

remove_stops_udf = udf(remove_stops, StringType())

# Define the columns to select from the DataFrame
pos_raw_cols = ["value"]

# Apply the UDF to remove stopwords
pos_rm_stops_df = pos_data_df.select(pos_raw_cols).withColumn("pos_stop_text", remove_stops_udf(pos_data_df["value"]))
pos_rm_stops_df.show(20)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


+--------------------+--------------------+
|               value|       pos_stop_text|
+--------------------+--------------------+
|[I can only say t...|[I say this: ee03...|
|[.maybe.Though I ...|[.maybe.Though I ...|
|[..well it doesn'...|[..well really on...|
|[and I haven't ev...|[and I even got f...|
|[ or will he just...|[ waste amounts m...|
|[ this is fine."<...|[ fine."<br /><br...|
|[ there was "Raw,...|[ "Raw," really s...|
|[ <br /><br />"Su...|[ <br /><br />"Su...|
|[although it was ...|[although actuall...|
|                  []|                  []|
|[.at least it wou...|[.at least would ...|
|[and the action s...|[and action shot ...|
|[but then again, ...|[but again, they'...|
|[he meant well. A...|[he meant well. A...|
|[but we watched i...|[but watched it, ...|
|[ just another un...|[ another unsolve...|
|[". After a coupl...|[". After couple ...|
|[but judge that f...|[but judge yourse...|
|[ but actually it...|[ actually kind a...|
|          [ kind of]|          

In [27]:
# Remove features from values
import re
import string
neg_rm_features_df = neg_rm_stops_df.select(neg_raw_cols+["neg_stop_text"]).withColumn("neg_feat_text", remove_features_udf(neg_rm_stops_df["neg_stop_text"]))
neg_rm_features_df.show(20)

+--------------------+--------------------+--------------------+
|               value|       neg_stop_text|       neg_feat_text|
+--------------------+--------------------+--------------------+
|[I can only say t...|[I say this: ee03...|  say this  portu...|
|[.maybe.Though I ...|[.maybe.Though I ...|maybe though  sor...|
|[..well it doesn'...|[..well really on...|well really one t...|
|[and I haven't ev...|[and I even got f...|and  even got far...|
|[ or will he just...|[ waste amounts m...|waste amounts mon...|
|[ this is fine."<...|[ fine."<br /><br...|fine   probably m...|
|[ there was "Raw,...|[ "Raw," really s...|raw really stand ...|
|[ <br /><br />"Su...|[ <br /><br />"Su...|   surprisingly m...|
|[although it was ...|[although actuall...|although actually...|
|                  []|                  []|                    |
|[.at least it wou...|[.at least would ...|  least would lik...|
|[and the action s...|[and action shot ...|and action shot  ...|
|[but then again, ...|[bu

In [28]:
# Remove features from values
import re
import string
pos_rm_features_df = pos_rm_stops_df.select(pos_raw_cols+["pos_stop_text"]).withColumn("pos_feat_text", remove_features_udf(pos_rm_stops_df["pos_stop_text"]))
pos_rm_features_df.show(20)

+--------------------+--------------------+--------------------+
|               value|       pos_stop_text|       pos_feat_text|
+--------------------+--------------------+--------------------+
|[I can only say t...|[I say this: ee03...|  say this  portu...|
|[.maybe.Though I ...|[.maybe.Though I ...|maybe though  sor...|
|[..well it doesn'...|[..well really on...|well really one t...|
|[and I haven't ev...|[and I even got f...|and  even got far...|
|[ or will he just...|[ waste amounts m...|waste amounts mon...|
|[ this is fine."<...|[ fine."<br /><br...|fine   probably m...|
|[ there was "Raw,...|[ "Raw," really s...|raw really stand ...|
|[ <br /><br />"Su...|[ <br /><br />"Su...|   surprisingly m...|
|[although it was ...|[although actuall...|although actually...|
|                  []|                  []|                    |
|[.at least it wou...|[.at least would ...|  least would lik...|
|[and the action s...|[and action shot ...|and action shot  ...|
|[but then again, ...|[bu

In [29]:
# Tagging text from value
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
neg_tagged_df = neg_rm_features_df.select(neg_raw_cols+["neg_feat_text"]).withColumn("neg_tagged_text", tag_and_remove_udf(neg_rm_features_df.neg_feat_text))
neg_tagged_df.show(20)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


+--------------------+--------------------+--------------------+
|               value|       neg_feat_text|     neg_tagged_text|
+--------------------+--------------------+--------------------+
|[I can only say t...|  say this  portu...| say portugal say...|
|[.maybe.Though I ...|maybe though  sor...| sort enjoyed sat...|
|[..well it doesn'...|well really one t...| judge house ramm...|
|[and I haven't ev...|and  even got far...| got infant terri...|
|[ or will he just...|waste amounts mon...| waste amounts mo...|
|[ this is fine."<...|fine   probably m...| fine credited op...|
|[ there was "Raw,...|raw really stand ...| raw stand materi...|
|[ <br /><br />"Su...|   surprisingly m...| murphy directed ...|
|[although it was ...|although actually...| wallach italian ...|
|                  []|                    |                    |
|[.at least it wou...|  least would lik...| least like think...|
|[and the action s...|and action shot  ...| action shot show...|
|[but then again, ...|but

In [30]:
# Tagging text from value
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
pos_tagged_df = pos_rm_features_df.select(pos_raw_cols+["pos_feat_text"]).withColumn("pos_tagged_text", tag_and_remove_udf(pos_rm_features_df.pos_feat_text))
pos_tagged_df.show(20)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


+--------------------+--------------------+--------------------+
|               value|       pos_feat_text|     pos_tagged_text|
+--------------------+--------------------+--------------------+
|[I can only say t...|  say this  portu...| say portugal say...|
|[.maybe.Though I ...|maybe though  sor...| sort enjoyed sat...|
|[..well it doesn'...|well really one t...| judge house ramm...|
|[and I haven't ev...|and  even got far...| got infant terri...|
|[ or will he just...|waste amounts mon...| waste amounts mo...|
|[ this is fine."<...|fine   probably m...| fine credited op...|
|[ there was "Raw,...|raw really stand ...| raw stand materi...|
|[ <br /><br />"Su...|   surprisingly m...| murphy directed ...|
|[although it was ...|although actually...| wallach italian ...|
|                  []|                    |                    |
|[.at least it wou...|  least would lik...| least like think...|
|[and the action s...|and action shot  ...| action shot show...|
|[but then again, ...|but

In [31]:
# Lemmatization for values
import re
import string
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
neg_lemm_df = neg_tagged_df.select(neg_raw_cols+["neg_tagged_text"]).withColumn("neg_lemm_text", lemmatize_udf(neg_tagged_df["neg_tagged_text"]))
neg_lemm_df.show(20)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


+--------------------+--------------------+--------------------+
|               value|     neg_tagged_text|       neg_lemm_text|
+--------------------+--------------------+--------------------+
|[I can only say t...| say portugal say...|say portugal say ...|
|[.maybe.Though I ...| sort enjoyed sat...|sort enjoy satani...|
|[..well it doesn'...| judge house ramm...|judge house ram b...|
|[and I haven't ev...| got infant terri...|get infant terrib...|
|[ or will he just...| waste amounts mo...|waste amount mone...|
|[ this is fine."<...| fine credited op...|fine credit open ...|
|[ there was "Raw,...| raw stand materi...|raw stand materia...|
|[ <br /><br />"Su...| murphy directed ...|murphy direct mov...|
|[although it was ...| wallach italian ...|wallach italian v...|
|                  []|                    |                    |
|[.at least it wou...| least like think...|least like think ...|
|[and the action s...| action shot show...|action shot show ...|
|[but then again, ...| go

In [32]:
# Lemmatization for values
import re
import string
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
pos_lemm_df = pos_tagged_df.select(pos_raw_cols+["pos_tagged_text"]).withColumn("pos_lemm_text", lemmatize_udf(pos_tagged_df["pos_tagged_text"]))
pos_lemm_df.show(20)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


+--------------------+--------------------+--------------------+
|               value|     pos_tagged_text|       pos_lemm_text|
+--------------------+--------------------+--------------------+
|[I can only say t...| say portugal say...|say portugal say ...|
|[.maybe.Though I ...| sort enjoyed sat...|sort enjoy satani...|
|[..well it doesn'...| judge house ramm...|judge house ram b...|
|[and I haven't ev...| got infant terri...|get infant terrib...|
|[ or will he just...| waste amounts mo...|waste amount mone...|
|[ this is fine."<...| fine credited op...|fine credit open ...|
|[ there was "Raw,...| raw stand materi...|raw stand materia...|
|[ <br /><br />"Su...| murphy directed ...|murphy direct mov...|
|[although it was ...| wallach italian ...|wallach italian v...|
|                  []|                    |                    |
|[.at least it wou...| least like think...|least like think ...|
|[and the action s...| action shot show...|action shot show ...|
|[but then again, ...| go

In [33]:
neg_check_blanks_df = neg_lemm_df.select(neg_raw_cols+["neg_lemm_text"]).withColumn("is_blank", check_blanks_udf(neg_lemm_df["neg_lemm_text"]))
# remove blanks from lemmatized text
neg_no_blanks_df = neg_check_blanks_df.filter(neg_check_blanks_df["is_blank"] == "False")

# drop duplicates from lemmatized text
neg_dedup_df = neg_no_blanks_df.dropDuplicates(['neg_lemm_text'])

neg_dedup_df.show(20)

+--------------------+--------------------+--------+
|               value|       neg_lemm_text|is_blank|
+--------------------+--------------------+--------+
|[ From then on, t...|action swift loca...|   False|
|[ Amitabh is 65 a...|        amitabh tabu|   False|
|[ for artistic pu...|artistic purpose ...|   False|
|[). I assume most...|assume actor gain...|   False|
|[ and all her bab...|babble go change ...|   False|
|[was there a scri...|be script villain...|   False|
|  [ become one with]|              become|   False|
|[ finally, but a ...|bit late get litt...|   False|
|[uh, bite film th...|bite film briefly...|   False|
|[ Boll.Lorenzo La...|boll lorenzo lama...|   False|
|[OK, the box look...|box look promise ...|   False|
|[. bulls**t! This...|bull film try any...|   False|
|[ in comparison w...|comparison compli...|   False|
|[..<br /><br />We...|crap act include ...|   False|
|[ well, all that ...|crap interest fre...|   False|
|[ but a completel...|different subject...|   

In [34]:
pos_check_blanks_df = pos_lemm_df.select(pos_raw_cols+["pos_lemm_text"]).withColumn("is_blank", check_blanks_udf(pos_lemm_df["pos_lemm_text"]))
# remove blanks from lemmatized text
pos_no_blanks_df = pos_check_blanks_df.filter(pos_check_blanks_df["is_blank"] == "False")

# drop duplicates from lemmatized text
pos_dedup_df = pos_no_blanks_df.dropDuplicates(['pos_lemm_text'])

pos_dedup_df.show(20)

+--------------------+--------------------+--------+
|               value|       pos_lemm_text|is_blank|
+--------------------+--------------------+--------+
|[ From then on, t...|action swift loca...|   False|
|[ Amitabh is 65 a...|        amitabh tabu|   False|
|[ for artistic pu...|artistic purpose ...|   False|
|[). I assume most...|assume actor gain...|   False|
|[ and all her bab...|babble go change ...|   False|
|[was there a scri...|be script villain...|   False|
|  [ become one with]|              become|   False|
|[ finally, but a ...|bit late get litt...|   False|
|[uh, bite film th...|bite film briefly...|   False|
|[ Boll.Lorenzo La...|boll lorenzo lama...|   False|
|[OK, the box look...|box look promise ...|   False|
|[. bulls**t! This...|bull film try any...|   False|
|[ in comparison w...|comparison compli...|   False|
|[..<br /><br />We...|crap act include ...|   False|
|[ well, all that ...|crap interest fre...|   False|
|[ but a completel...|different subject...|   

In [35]:
# Rename final column as text for processing
pos_final_df = pos_dedup_df.selectExpr("pos_lemm_text as text")
pos_final_df.show()
pos_final_df.printSchema()

+--------------------+
|                text|
+--------------------+
|open smell urine ...|
|star frothy roman...|
|rest actor poor s...|
|babble go change ...|
|first time hop ma...|
|undemanding way s...|
|least like think ...|
|different subject...|
|be script villain...|
|matter element ma...|
|wet bandit wait l...|
|think see worst m...|
|crap act include ...|
|give dunno effort...|
|don think look an...|
|film make documen...|
|doesn cliff hanga...|
|bite film briefly...|
|uninteresting peo...|
|judge house ram b...|
+--------------------+
only showing top 20 rows

root
 |-- text: string (nullable = true)



In [36]:
# Rename final column as text for processing
neg_final_df = neg_dedup_df.selectExpr("neg_lemm_text as text")
neg_final_df.show()
neg_final_df.printSchema()

+--------------------+
|                text|
+--------------------+
|open smell urine ...|
|star frothy roman...|
|rest actor poor s...|
|babble go change ...|
|first time hop ma...|
|undemanding way s...|
|least like think ...|
|different subject...|
|be script villain...|
|matter element ma...|
|wet bandit wait l...|
|think see worst m...|
|crap act include ...|
|give dunno effort...|
|don think look an...|
|film make documen...|
|doesn cliff hanga...|
|bite film briefly...|
|uninteresting peo...|
|judge house ram b...|
+--------------------+
only showing top 20 rows

root
 |-- text: string (nullable = true)



In [60]:
from pyspark.sql.functions import monotonically_increasing_id
# Create Unique ID for text
neg_uid_df = neg_final_df.withColumn("neg_uid", monotonically_increasing_id())
neg_uid_df.show(4)

+--------------------+-------+
|                text|neg_uid|
+--------------------+-------+
|open smell urine ...|      0|
|star frothy roman...|      1|
|rest actor poor s...|      2|
|babble go change ...|      3|
+--------------------+-------+
only showing top 4 rows



In [61]:
from pyspark.sql.functions import monotonically_increasing_id
# Create Unique ID for text
pos_uid_df = pos_final_df.withColumn("pos_uid", monotonically_increasing_id())
pos_uid_df.show(4)

+--------------------+-------+
|                text|pos_uid|
+--------------------+-------+
|open smell urine ...|      0|
|star frothy roman...|      1|
|rest actor poor s...|      2|
|babble go change ...|      3|
+--------------------+-------+
only showing top 4 rows



In [62]:
# Create label with 1.0 being the constant for ML classifier
from pyspark.sql.functions import udf

@udf("double")
def const_col():
    return 1.0

neg_label_df = neg_uid_df.withColumn('label', const_col())
neg_label_df.show()

+--------------------+-------+-----+
|                text|neg_uid|label|
+--------------------+-------+-----+
|open smell urine ...|      0|  1.0|
|star frothy roman...|      1|  1.0|
|rest actor poor s...|      2|  1.0|
|babble go change ...|      3|  1.0|
|first time hop ma...|      4|  1.0|
|undemanding way s...|      5|  1.0|
|least like think ...|      6|  1.0|
|different subject...|      7|  1.0|
|be script villain...|      8|  1.0|
|matter element ma...|      9|  1.0|
|wet bandit wait l...|     10|  1.0|
|think see worst m...|     11|  1.0|
|crap act include ...|     12|  1.0|
|give dunno effort...|     13|  1.0|
|don think look an...|     14|  1.0|
|film make documen...|     15|  1.0|
|doesn cliff hanga...|     16|  1.0|
|bite film briefly...|     17|  1.0|
|uninteresting peo...|     18|  1.0|
|judge house ram b...|     19|  1.0|
+--------------------+-------+-----+
only showing top 20 rows



In [63]:
# Create label with 1.0 being the constant for ML classifier
from pyspark.sql.functions import udf

@udf("double")
def const_col():
    return 1.0

pos_label_df = pos_uid_df.withColumn('label', const_col())
pos_label_df.show()

+--------------------+-------+-----+
|                text|pos_uid|label|
+--------------------+-------+-----+
|open smell urine ...|      0|  1.0|
|star frothy roman...|      1|  1.0|
|rest actor poor s...|      2|  1.0|
|babble go change ...|      3|  1.0|
|first time hop ma...|      4|  1.0|
|undemanding way s...|      5|  1.0|
|least like think ...|      6|  1.0|
|different subject...|      7|  1.0|
|be script villain...|      8|  1.0|
|matter element ma...|      9|  1.0|
|wet bandit wait l...|     10|  1.0|
|think see worst m...|     11|  1.0|
|crap act include ...|     12|  1.0|
|give dunno effort...|     13|  1.0|
|don think look an...|     14|  1.0|
|film make documen...|     15|  1.0|
|doesn cliff hanga...|     16|  1.0|
|bite film briefly...|     17|  1.0|
|uninteresting peo...|     18|  1.0|
|judge house ram b...|     19|  1.0|
+--------------------+-------+-----+
only showing top 20 rows



In [64]:
pos_data = pos_label_df.select('pos_uid','text','label')
pos_data.show(20)

+-------+--------------------+-----+
|pos_uid|                text|label|
+-------+--------------------+-----+
|      0|open smell urine ...|  1.0|
|      1|star frothy roman...|  1.0|
|      2|rest actor poor s...|  1.0|
|      3|babble go change ...|  1.0|
|      4|first time hop ma...|  1.0|
|      5|undemanding way s...|  1.0|
|      6|least like think ...|  1.0|
|      7|different subject...|  1.0|
|      8|be script villain...|  1.0|
|      9|matter element ma...|  1.0|
|     10|wet bandit wait l...|  1.0|
|     11|think see worst m...|  1.0|
|     12|crap act include ...|  1.0|
|     13|give dunno effort...|  1.0|
|     14|don think look an...|  1.0|
|     15|film make documen...|  1.0|
|     16|doesn cliff hanga...|  1.0|
|     17|bite film briefly...|  1.0|
|     18|uninteresting peo...|  1.0|
|     19|judge house ram b...|  1.0|
+-------+--------------------+-----+
only showing top 20 rows



In [65]:
neg_data = neg_label_df.select('neg_uid','text','label')
neg_data.show(20)

+-------+--------------------+-----+
|neg_uid|                text|label|
+-------+--------------------+-----+
|      0|open smell urine ...|  1.0|
|      1|star frothy roman...|  1.0|
|      2|rest actor poor s...|  1.0|
|      3|babble go change ...|  1.0|
|      4|first time hop ma...|  1.0|
|      5|undemanding way s...|  1.0|
|      6|least like think ...|  1.0|
|      7|different subject...|  1.0|
|      8|be script villain...|  1.0|
|      9|matter element ma...|  1.0|
|     10|wet bandit wait l...|  1.0|
|     11|think see worst m...|  1.0|
|     12|crap act include ...|  1.0|
|     13|give dunno effort...|  1.0|
|     14|don think look an...|  1.0|
|     15|film make documen...|  1.0|
|     16|doesn cliff hanga...|  1.0|
|     17|bite film briefly...|  1.0|
|     18|uninteresting peo...|  1.0|
|     19|judge house ram b...|  1.0|
+-------+--------------------+-----+
only showing top 20 rows



In [ ]:
# Split the data into training and test sets (30% held out for testing)
(pos_trainingData, pos_testData) = pos_data.randomSplit([0.7, 0.3])

In [ ]:
# Split the data into training and test sets (40% held out for testing)
(neg_trainingData, neg_testData) = neg_data.randomSplit([0.7, 0.3])

In [ ]:
# Running Naive Bayes classifier.
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and nb.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
#data = tokenizer.transform(data)
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="rawFeatures")
labelEncoder = StringIndexer(inputCol="text", outputCol='label')
#vectorizer = CountVectorizer(inputCol= "words", outputCol="rawFeatures")
idf = IDF(minDocFreq=2000, inputCol="rawFeatures", outputCol="features")
#assembler = VectorAssembler(inputCols=["hour", "mobile", "userFeatures"],outputCol="features")
#idfModel = idf.fit(data)

#lda = LDA(k=20, seed=1, optimizer="em")

# DecisionTree Classifier
#dt = DecisionTreeClassifier()

# Linear Support Vector Classifer
#lsvc = LinearSVC((maxIter=10, regParam=0.1)

# Naive Bayes model
nb = NaiveBayes(smoothing=2.0)

# Random Forest Classifier
#rfc = RandomForestClassifier()

# Pipeline Architecture for NB
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, nb])

# Pipeline Architecture for RFC
#pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, rfc])

# Pipeline Architecture for DT
#pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, dt])

# Pipeline Architecture for LSVC
#pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, lsvc])

In [ ]:
# Train model.  This also runs the indexers.
neg_model = pipeline.fit(neg_trainingData)
neg_model


In [ ]:
# Train model.  This also runs the indexers.
pos_model = pipeline.fit(pos_trainingData)
pos_model


In [ ]:
pos_predictions = pos_model.transform(pos_testData)

# Select example rows to display.
pos_predictions.select("text", "label", "prediction").show(5,True)

In [ ]:
neg_predictions = neg_model.transform(neg_testData)

# Select example rows to display.
neg_predictions.select("text", "label", "prediction").show(5,True)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
neg_evaluator = RegressionEvaluator(predictionCol="prediction")
neg_evaluator.evaluate(neg_predictions)

In [51]:
from pyspark.ml.evaluation import RegressionEvaluator
pos_evaluator = RegressionEvaluator(predictionCol="prediction")
pos_evaluator.evaluate(pos_predictions)

1.0

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
pos_evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
pos_evaluator.evaluate(pos_predictions)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
neg_evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
neg_evaluator.evaluate(neg_predictions)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
pos_evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
pos_evaluator.evaluate(pos_predictions)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
neg_evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
neg_evaluator.evaluate(neg_predictions)

In [ ]:
from pyspark.sql.types import FloatType

from textblob import TextBlob

def sentiment_analysis(text):
    return TextBlob(text).sentiment.polarity

sentiment_analysis_udf = udf(sentiment_analysis , FloatType())

In [ ]:
df = neg_label_df.withColumn("sentiment_score", sentiment_analysis_udf(neg_label_df['text'] ))
df.show(20,True)

In [ ]:
df2 = pos_label_df.withColumn("sentiment_score", sentiment_analysis_udf(pos_label_df['text'] ))
df2.show(20,True)

In [ ]:
def condition(r):
    if (r >=0.1):
        label = "positive"
    elif(r <= -0.1):
        label = "negative"
    else:
        label = "neutral"
    return label

sentiment_udf = udf(lambda x: condition(x), StringType())